# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Bring in data from csv
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data


,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,0,manggar,ID,100,1603027192,81,-2.88,108.27,81.21,10.36
1,1,pangnirtung,CA,100,1603027192,77,66.15,-65.71,35.62,7.07
2,2,xiongzhou,CN,59,1603027192,82,25.12,114.30,61.57,2.86
3,3,puerto ayora,EC,100,1603027192,89,-0.74,-90.35,70.00,5.99
4,4,kapaa,US,1,1603027193,94,22.08,-159.32,73.40,4.70
...,...,...,...,...,...,...,...,...,...,...
535,576,saint-augustin,CA,99,1603027364,68,51.23,-58.65,47.80,18.10
536,577,hami,CN,100,1603027364,36,42.80,93.45,52.07,5.41
537,578,kangundo,KE,75,1603027209,53,-1.30,37.35,75.20,9.17
538,579,kampot,KH,75,1603027365,94,10.62,104.18,77.00,4.90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [12]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Set intensity
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Windspeed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Windspeed
87,95,chuy,UY,0,1603027214,65,-33.70,-53.46,71.33,5.84
176,191,tura,RU,0,1603027245,91,25.52,90.22,75.25,2.08
198,216,faanui,PF,0,1603027249,81,-16.48,-151.75,78.85,7.29
224,242,viedma,AR,0,1603027263,28,-40.81,-63.00,70.00,3.00
244,262,soller,ES,0,1603027265,37,39.77,2.72,72.00,5.82
264,284,hamadan,IR,0,1603027279,9,34.80,48.51,73.40,4.70
405,435,esfarayen,IR,0,1603027103,1,37.08,57.51,75.20,4.70
467,501,javanrud,IR,0,1603027345,15,34.80,46.52,75.16,3.67
481,517,bojnurd,IR,0,1603027347,3,37.47,57.33,73.40,2.24
485,521,along,IN,0,1603027348,87,28.17,94.77,71.24,1.43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
87,chuy,UY,-33.70,-53.46,
176,tura,RU,25.52,90.22,
198,faanui,PF,-16.48,-151.75,
224,viedma,AR,-40.81,-63.00,
244,soller,ES,39.77,2.72,
264,hamadan,IR,34.80,48.51,
405,esfarayen,IR,37.08,57.51,
467,javanrud,IR,34.80,46.52,
481,bojnurd,IR,37.47,57.33,
485,along,IN,28.17,94.77,


In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [17]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 87: chuy.
Closest hotel in chuy is Bertelli Chuí Hotel.
------------
Retrieving Results for Index 176: tura.
Closest hotel in tura is Hotel Polo Orchid.
------------
Retrieving Results for Index 198: faanui.
Closest hotel in faanui is InterContinental Bora Bora Resort Thalasso Spa.
------------
Retrieving Results for Index 224: viedma.
Closest hotel in viedma is Hotel Austral Viedma.
------------
Retrieving Results for Index 244: soller.
Closest hotel in soller is Bikini Island & Mountain Hotels.
------------
Retrieving Results for Index 264: hamadan.
Closest hotel in hamadan is Parsian BuAli Hotel.
------------
Retrieving Results for Index 405: esfarayen.
Closest hotel in esfarayen is مهمانپذیر آفتاب.
------------
Retrieving Results for Index 467: javanrud.
Closest hotel in javanrud is هتل چیا.
------------
Retrieving Results for Index 481: bojnurd.
Closest hotel in bojnurd is Hotel negin.
------------
Retrieving Results for Index 485: along.
Closest hotel

In [18]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
87,chuy,UY,-33.70,-53.46,Bertelli Chuí Hotel
176,tura,RU,25.52,90.22,Hotel Polo Orchid
198,faanui,PF,-16.48,-151.75,InterContinental Bora Bora Resort Thalasso Spa
224,viedma,AR,-40.81,-63.00,Hotel Austral Viedma
244,soller,ES,39.77,2.72,Bikini Island & Mountain Hotels
264,hamadan,IR,34.80,48.51,Parsian BuAli Hotel
405,esfarayen,IR,37.08,57.51,مهمانپذیر آفتاب
467,javanrud,IR,34.80,46.52,هتل چیا
481,bojnurd,IR,37.47,57.33,Hotel negin
485,along,IN,28.17,94.77,Hotel West


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Figure
fig


Figure(layout=FigureLayout(height='420px'))